In [32]:
#импорт библиотек
import cv2
import numpy as np
import math

#размеры окна
WIND_X = 600
WIND_Y = 500

g = 0.3

#базовый класс
class Figure:
    def __init__ (self, x_, y_, vx_, vy_):
        self.x  = x_
        self.y  = y_
        self.vx = vx_
        self.vy = vy_

    def draw (self, canvas):
        pass
    
    def move (self):
        self.x += self.vx
        self.y += self.vy
        
        if (self.x < 0 or self.x > WIND_X):
            self.vx *= -1

        if (self.y < 0 or self.y > WIND_Y):
            self.vy *= -1

#производный класс Круг
class Circle (Figure):
    def __init__ (self, x_, y_, vx_, vy_, r_, color_by_coords_, ay_ = g):
        Figure.__init__ (self, x_, y_, vx_, vy_)
        
        self.r = r_
        self.ay = ay_
        self.color_by_coords = color_by_coords_
        
    def draw (self, canvas):
        color = self.color_by_coords (self.x, self.y, self.vx, self.vy)
        
        cv2.circle (canvas, (self.x, self.y), self.r, color, 4)
    
    def move (self):
        self.vy += self.ay

        if (self.x < self.r or self.x > WIND_X - self.r):
            self.vx *= -1

        if (self.y < self.r or self.y > WIND_Y - self.r):
            self.vy *= -1
        
        self.x += int (self.vx)
        self.y += int (self.vy)
        
#производный класс Квадрат
class Square (Figure):
    def __init__ (self, x_, y_, vx_, vy_, sz_):
        Figure.__init__ (self, x_, y_, vx_, vy_)
        
        self.sz = sz_
        
    def draw (self, canvas):
        cv2.rectangle (canvas, (self.x, self.y), (self.x + self.sz, self.y + self.sz),
            (10, 120, 50), -1)
    
    def move (self):
        self.x += self.vx
        self.y += self.vy
        
        if (self.x < 0 or self.x > WIND_X - self.sz):
            self.vx *= -1

        if (self.y < 0 or self.y > WIND_Y - self.sz):
            self.vy *= -1

#производный класс Линия
class Line (Figure):
    def __init__ (self, x1_, y1_, vx1_, vy1_, x2_, y2_, vx2_, vy2_):
        Figure.__init__ (self, x1_, y1_, vx1_, vy1_)
        
        self.fig1 = Figure (x1_, y1_, vx1_, vy1_)
        self.fig2 = Figure (x2_, y2_, vx2_, vy2_)
        
    def draw (self, canvas):
        cv2.line (canvas, (self.fig1.x, self.fig1.y),
            (self.fig2.x, self.fig2.y),
            ((255 + int (self.fig1.x * 0.3)) % 255,
             (210 + int (self.fig2.y * 0.56)) % 255, 130 +
             np.random.randint (5, 20)), 2)
    
    def move (self):
        self.fig1.move ()
        self.fig2.move ()

#менеджер
class Manager:
    def __init__ (self, canvas_ref_):
        #лист со всеми фигурами
        self.figures = []
        
        self.refresh_speed = 5
        self.pen_coord = 10
        self.graph_canvas = canvas_ref_.copy ()
        
        self.Ek = 0
        self.Ep = 0
    
    def move (self):
        #движение
        for fig in self.figures:
            fig.move ()

    def draw (self, canvas):
        #рисование
        for fig in self.figures:
            fig.draw (canvas)

    def add_figure (self, figure):
        self.figures.append (figure)
    
    def handle_collisions (self):
        for i in range (len (self.figures)):
            for j in range (i + 1, len (self.figures)):
                fig1 = self.figures [i]
                fig2 = self.figures [j]
                
                distance = math.sqrt ((fig1.x - fig2.x)**2 + (fig1.y - fig2.y)**2)
                
                if (distance <= fig1.r + fig2.r):
                    vx1     = fig1.vx
                    fig1.vx = fig2.vx
                    fig2.vx = vx1
                    
                    vy1     = fig1.vy
                    fig1.vy = fig2.vy
                    fig2.vy = vy1
    
    def visualize_energy (self, scale = 3):
        self.pen_coord += self.refresh_speed
        
        if (self.pen_coord >= WIND_X - 10):
            self.pen_coord = 10
        
        cv2.circle (self.graph_canvas, (self.pen_coord, int (WIND_Y - self.Ek * scale)),
                    3, (100, 200, 0), -1)

        cv2.circle (self.graph_canvas, (self.pen_coord, int (WIND_Y - self.Ep * scale)),
                    3, (200, 20, 0), -1)

        cv2.circle (self.graph_canvas, (self.pen_coord,
            int (WIND_Y - (self.Ek + self.Ep) * scale)), 3, (255, 255, 255), -1)
        
        return self.graph_canvas
    
    def calc_energy (self):
        self.Ek = 0
        self.Ep = 0
        
        for fig in self.figures:
            self.Ek += (fig.vx**2 + fig.vy**2) / 2
            self.Ep += g * (WIND_Y - fig.r - fig.y)
        
        print ("Ek, Ep: ", self.Ek, self.Ep)

def color_by_coord_CHAOTIC_EVIL (x, y, vx, vy):
    return ((x * 3) % 255,
            (y * 4 + vx * 10) % 255,
            (vy * 20 + 100) % 255)

def color_by_coord_shades_of_gray (x, y, vx, vy):
    return ((y + vx * 6) % 255,
            (y + vx * 5) % 255,
            (y + vx * 4) % 255)

#холст
canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 60

manager = Manager (canvas)

#добавление новой
#figures.append (Circle (100, 200, 3, 5, 50))
#figures.append (Square (300, 200, -1, 3, 60))
#figures.append (Line   (300, 200, -1, 3, 400, 100, 2, 5))

#цикл (основная часть программы)
while (True):
    #canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 200
    
    #движение
    manager.move ()
    manager.handle_collisions ()
    manager.calc_energy ()
    
    #визуализация
    manager.draw (canvas)

    #вывод
    graphs = manager.visualize_energy ()
    frame = np.concatenate ((canvas, graphs), axis=1)
    
    cv2.imshow ("frame", frame)
    
    #обработка клавиатуры
    key = cv2.waitKey (1) & 0xFF
    
    if (key == ord ('q')):
        break
    
    vx1 = np.random.randint (3, 7)
    vy1 = np.random.randint (3, 7)
        
    if (key == ord ('a')):
        manager.add_figure (Circle (300, 200, vx1, vy1,
            60, color_by_coord_CHAOTIC_EVIL))

    if (key == ord ('s')):
        manager.add_figure (Circle (300, 200, vx1, vy1,
            60, color_by_coord_shades_of_gray))


#закрытие окон
cv2.destroyAllWindows ()

Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  32.345 70.2
Ek, Ep:  34.28 68.39999999999999
Ek, Ep:  36.30499999999999 66.6
Ek, Ep:  38.419999999999995 64.5
Ek, Ep:  40.62499999999999 62.4
Ek, Ep:  42.91999999999999 60.3
Ek, Ep:  45.305 57.9
Ek, Ep:  47.78 55.5
Ek, Ep:  50.345000000000006 53.1
Ek, Ep:  53.000000000000014 50.4
Ek, Ep:  55.745000000000026 47.699999999999996
Ek, Ep:  58.580000000000034 45.0
Ek, Ep:  61.50500000000004 42.3
Ek, Ep:  64.52000000000004 39.3
Ek, Ep:  67.62500000000006 36.3
Ek, Ep:  70.82000000000006 33.3
Ek, Ep: 

Ek, Ep:  77.48000000000009 42.6
Ek, Ep:  74.10500000000008 45.9
Ek, Ep:  70.82000000000006 48.9
Ek, Ep:  67.62500000000006 51.9
Ek, Ep:  64.52000000000004 54.9
Ek, Ep:  61.50500000000004 57.599999999999994
Ek, Ep:  58.580000000000034 60.3
Ek, Ep:  55.745000000000026 63.0
Ek, Ep:  53.000000000000014 65.7
Ek, Ep:  50.345000000000006 68.1
Ek, Ep:  47.78 70.5
Ek, Ep:  45.305 72.89999999999999
Ek, Ep:  42.92 75.0
Ek, Ep:  40.625 77.1
Ek, Ep:  38.42 79.2
Ek, Ep:  36.30500000000001 81.0
Ek, Ep:  34.28 82.8
Ek, Ep:  32.345000000000006 84.6
Ek, Ep:  30.500000000000004 86.39999999999999
Ek, Ep:  28.745000000000005 87.89999999999999
Ek, Ep:  27.080000000000005 89.39999999999999
Ek, Ep:  25.50500000000001 90.89999999999999
Ek, Ep:  24.02000000000001 92.1
Ek, Ep:  22.625000000000007 93.3
Ek, Ep:  21.320000000000007 94.5
Ek, Ep:  20.105000000000008 95.39999999999999
Ek, Ep:  18.980000000000008 96.3
Ek, Ep:  17.945000000000007 97.2
Ek, Ep:  17.000000000000007 98.1
Ek, Ep:  16.145000000000007 98.7
Ek,

Ek, Ep:  14.119999999999994 108.89999999999999
Ek, Ep:  14.704999999999995 108.3
Ek, Ep:  15.379999999999992 107.7
Ek, Ep:  16.14499999999999 107.1
Ek, Ep:  16.99999999999999 106.5
Ek, Ep:  17.944999999999986 105.6
Ek, Ep:  18.979999999999986 104.7
Ek, Ep:  20.104999999999983 103.8
Ek, Ep:  21.319999999999983 102.6
Ek, Ep:  22.62499999999998 101.39999999999999
Ek, Ep:  24.01999999999998 100.2
Ek, Ep:  25.504999999999974 98.7
Ek, Ep:  27.079999999999973 97.2
Ek, Ep:  28.74499999999997 95.7
Ek, Ep:  30.499999999999968 94.2
Ek, Ep:  32.34499999999997 92.39999999999999
Ek, Ep:  34.27999999999996 90.6
Ek, Ep:  36.304999999999964 88.8
Ek, Ep:  38.41999999999996 86.7
Ek, Ep:  40.62499999999996 84.6
Ek, Ep:  42.919999999999945 82.5
Ek, Ep:  45.30499999999996 80.1
Ek, Ep:  47.77999999999996 77.7
Ek, Ep:  50.34499999999996 75.3
Ek, Ep:  52.99999999999997 72.89999999999999
Ek, Ep:  55.744999999999976 70.2
Ek, Ep:  58.57999999999998 67.5
Ek, Ep:  61.50499999999999 64.8
Ek, Ep:  64.52 61.8
Ek, Ep: 